In [ ]:
from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import Conv1D, MaxPooling1D
from keras.datasets import imdb
import tensorflow as tf
import numpy as np

In [ ]:
#Allocation of memory for the GPU process
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_virtual_device_configuration(gpus[0],
      [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=3072)])
logical = tf.config.experimental.list_logical_devices('GPU')
print(logical[0])

In [ ]:
# Embedding
max_features = 20000
maxlen = 100
embedding_size = 128

# Convolution
kernel_size = 5
filters = 64
pool_size = 4

# LSTM
lstm_output_size = 70

# Training
batch_size = 30
epochs = 100 #modify as required, should be even

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')


print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

#Data split for unsupervised training
x_train1 = x_train[0:5000]
x_train2 = x_train[5001:24999]
y_train1 = y_train[0:5000]
y_train2 = y_train[5001:24999]


In [ ]:
print('Build models...')

class Network(Sequential):
  def __init__(self):
    super().__init__()
    self.add(Embedding(max_features, embedding_size, input_length=maxlen))
    self.add(Dropout(0.25))
    self.add(Conv1D(filters, kernel_size, padding='valid', activation='relu', strides=1))
    self.add(MaxPooling1D(pool_size=pool_size))
    self.add(LSTM(lstm_output_size))
    self.add(Dense(1))
    self.add(Activation('sigmoid'))

#Default for starting weights
model = Network()
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model_benchmark = Network()
model_small_set = Network()
model_iterative_training = Network()


In [ ]:
#Benchmark
model_benchmark.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model_benchmark.set_weights(model.get_weights())

print('Train Benchmark')
model_benchmark.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test),verbose=2)
score, acc = model.evaluate(x_test, y_test, batch_size=batch_size)
print('Benchmark test score:', score)
print('Benchmark test accuracy:', acc)
model_benchmark.save('Mdl_Bench.h5')

In [ ]:
#Smaller training set
print('Train on smaller set')
model_small_set.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model_small_set.set_weights(model.get_weights())
model_small_set.fit(x_train1, y_train1,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test),verbose=2)
score, acc = model_small_set.evaluate(x_test, y_test, batch_size=batch_size)
print('Small model test score:', score)
print('Small model test accuracy:', acc)
model_small_set.save('Mdl_Small.h5')

In [ ]:
#Iterative_train
print('Train iter model')
model_iterative_training.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model_iterative_training.set_weights(model.get_weights())

model_iterative_training.fit(x_train1, y_train1,
          batch_size=batch_size,
          epochs=epochs/2,
          validation_data=(x_test, y_test),verbose=2)

a = model_iterative_training.predict(x_train2)

i = 0
k = np.array([1])
for x in np.nditer(a):
     if x>=0.5:
        x_train1 = np.append(x_train1, np.array([x_train2[i]]), axis = 0)
        y_train1 = np.append(y_train1, k)
        i += 1
     else:
        i += 1


model_iterative_training.fit(x_train1, y_train1,
          batch_size=batch_size,
          epochs=epochs/2,
          validation_data=(x_test, y_test),verbose=2)

score, acc = model_iterative_training.evaluate(x_test, y_test, batch_size=batch_size, verbose =2)
print('Iterative test score:', score)
print('Iterative test accuracy:', acc)
model_iterative_training.save('Mdl_iter.h5')